# Story from economic perspective

In [42]:
%run set_theme.ipynb

In [43]:
# Imports
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import statsmodels.api as sm

In [44]:
# Datasets
ua_scores_df = pd.read_csv('../data/uaScoresDataFrame.csv')
train_df = pd.read_csv('../data/train.csv')
healthy_lifestyle_df = pd.read_csv('../data/healthy_lifestyle_city_2021.csv')

In [45]:
#cleaning
ua_scores_df.columns = ua_scores_df.columns.str.strip().str.replace(' ', '_').str.lower()

In [46]:
european_cities = [
    "London",
    "Paris",
    "Berlin",
    "Amsterdam",
    "Barcelona",
    "Lisbon",
    "Rome",
    "Athens",
    "Vienna",
    "Dublin",
    "Copenhagen",
    "Stockholm",
    "Oslo",
    "Helsinki",
    "Prague",
    "Budapest",
    "Warsaw",
    "Sofia",
    "Belgrade",
]

## The well being of European citizens

![Happy people](../static/happy_people.jpeg)

The quest to enhance quality of life in urban settings has been a focal point of societal and political discourse. Despite considerable progress, numerous factors continue to influence the well-being of citizens. These factors, ranging from economic conditions to environmental sustainability, shape the daily experiences and happiness levels of urban residents.
Various studies have highlighted different factors that influence well-being in cities, such as income, air quality, green spaces, safety, healthcare, and education (Xu et al., 2023). Some argue that economic prosperity is the key to happiness, while others emphasize environmental and social factors (Livingston et al., 2022). In our exploration, we delve into two primary perspectives: the economic lens, which posits that prosperity correlates with happiness, and the sustainability perspective, which emphasizes environmental and social well-being. 

This data story focuses on exploring the well-being of people in cities by examining a range of influencing factors from these two perspectives. Data from multiple sources, including the primary source "City Happiness Index - 2024," will be used to provide insights (City Happiness Index - 2024, 2024). The City Happiness Index is a comprehensive dataset that measures various aspects of urban well-being across multiple cities worldwide. We will specifically look at the relationship between well-being and factors such as income compared to happiness, the happiness score and air quality index by city, the amount of green space and happiness score by city, pollution levels, and their impact on well-being in cities.
By analyzing these factors, we aim to understand how they contribute to the overall well-being of city residents. Specific metrics, such as the happiness score, air quality index, and pollution levels, will be defined and calculated to provide a clear picture of well-being in various urban settings. By comparing these factors across different cities, we hope to identify patterns and provide recommendations for improving urban well-being.

## Wealthy cities

It is logical that living in a prosperous city is more enjoyable, and this can be attributed to several key factors. Prosperous cities often provide better access to things like high-quality healthcare and education, as can be seen in the graph below

In [47]:
ua_scores_df['ua_country'] = ua_scores_df['ua_country'].str.strip()

countries = ['Netherlands', 'Sweden', 'Norway', 'Serbia', 'Bulgaria', 'Slovakia']

subset_df_big = ua_scores_df[ua_scores_df['ua_country'].isin(countries)]

fig_bubble = px.scatter(
    subset_df_big,
    x='healthcare',
    y='education',
    size='economy',
    color='ua_country',
    hover_name='ua_name',
    title='Healthcare and Education Scores vs Economy',
    labels={
        'ua_country': 'Country',
        'healthcare': 'Healthcare',
        'education': 'Education',
        'economy': 'Economy Score'
    },
    size_max=25
)

fig_bubble.update_layout(
    xaxis_title='Healthcare',
    yaxis_title='Education',
)

fig_bubble.add_annotation(
                    x=0, y=-0.2,
                    xref='paper',
                    yref='paper',
                    showarrow=False,
                    xanchor='left',
                    yanchor='bottom',
                    align='left',
                    text='The size of the bubble corresponds to the economy of that city.', 
)

fig_bubble.show()

Well-funded healthcare systems ensure that residents receive timely and effective medical care, leading to better health outcomes and less stress related to health issues. Similarly, excellent educational institutions offer opportunities for personal and professional growth, fostering a sense of accomplishment and future security. Better educational institutions often lead to better employment opportunities. Higher wages, job security, and career advancement prospects contribute to financial stability, reducing anxiety about meeting basic needs and allowing individuals to focus on personal fulfillment and family life. But does this mean that people are actually happier in a prosperous city?

In [48]:
ua_renamed_df = ua_scores_df.rename(columns={'ua_name': 'City'})
merged_ua_train_df = pd.merge(train_df, ua_renamed_df, on='City')

subset_df = merged_ua_train_df[merged_ua_train_df['City'].isin(['Belgrade', 'Amsterdam', 'Sofia'])]

# Radar chart
fig_radar = go.Figure()

for _, row in subset_df.iterrows():
    fig_radar.add_trace(go.Scatterpolar(
        r=[row['healthcare'], row['education'], row['economy'], row['Happiness_Score']],
        theta=['Healthcare', 'Education', 'Economy', 'Happiness'],
        fill='toself',
        name=row['City']
    ))

fig_radar.update_layout(
    polar=dict(
        radialaxis=dict(visible=True)
    ),
    title='Healthcare, Happiness and Education Scores vs Economy zoomed in'
)

fig_radar.show()

As can be seen from the graph above, it is indeed true that people in an affluent city, who have better fascilities like healthcare and education also indeed score better when it comes to happiness as seen with Amsterdam which is highlighted in de graph above. The combination of better education, healthcare, and living conditions in prosperous cities seems to create a conducive environment for happiness. Within wealthier cities there are more opportunities to invest in things like education and healthcare. These investments tend to foster conditions that support individual happiness and overall well-being.

Another key indicator for economic wealth of a city is its profit, as it encapsulates the success and health of local businesses, the well-being of the workforce, and the strength of the city's economic infrastructure. A profitable business environment fosters a cycle of growth, investment, and improvement, ensuring long-term economic stability and prosperity for the city's residents.

In [49]:
renamed_train_df = train_df.rename(columns={'City': 'city'})
best_cities_df = pd.read_csv('../data/GreenCities-Data.csv')

In [50]:
merged_best_train_df = pd.merge(renamed_train_df, best_cities_df, on='city')

best_train_filtered_df = merged_best_train_df[merged_best_train_df['city'].isin(european_cities)]

best_train_filtered_df.head()

profit = best_train_filtered_df['Profit']
happiness_score = best_train_filtered_df['Happiness_Score']

# Fit the regression line using statsmodels
X = sm.add_constant(80 - profit)  # Adding a constant for the intercept
model = sm.OLS(happiness_score, X).fit()
trendline = model.predict(X)

# Create scatter plot with different colors for each city
scatter_plot = px.scatter(
    best_train_filtered_df,
    x=(80 - profit),
    y='Happiness_Score',
    color='city',
    hover_name='city',
    labels={'city': 'City'},
    title='Profit in comparison to Happiness',
)

# Add the regression line to the scatter plot
scatter_plot.add_trace(
    go.Scatter(
        x=(80 - profit),
        y=trendline,
        mode='lines',
        name='Trendline',
        line=dict(color='#4c75eb'),
    )
)

scatter_plot.update_traces(marker=dict(size=15))

scatter_plot.update_layout(
    xaxis_title='Profit',
    yaxis_title='Happiness Score'
)

scatter_plot.show()

As seen in the given graph a clear correlation between profit and happiness emerges. Understanding what contributes to urban residents' happiness reveals a complex interplay of factors. Cities with higher profits as measured by ease of doing business, GDP, and tourism generally give high happiness scores - these include London, Paris, and Vienna. The term 'ease of doing business' refers to any city that is conducive to starting or running businesses. On the other hand, GDP indicates economic output and wealth, while tourism measures the city's ability to attract visitors to the local economy and cultural exchange. These factors lead to more job opportunities, associated with diverse cultural activities, enhancing residents' happiness. Cities doing better economically tend to have more jobs, better infrastructure, and more recreational and cultural amenities, all of which contribute to wellbeing.
The results underscore the role of economic policies and urban planning in providing a base for both financial prosperity and holistic well-being. Economic development should be counterpoised with cultural vitality, full community engagement, and better quality of life. This is the way through which cities can thrive economically while maximizing happiness and well-being among their residents.